#                                         Negassi Tesfay
##                                                 Airbnb Data Challenge



To accomplish this challenge, the process is divided into three major tasks: <br>
> Data Wrangling.<br>
> Exploratory Data Analysis.<br>
> Predictive Modeling <br>

And each task is done into separate files and this files is for __Data Wrangling__.<br>

Data wringling: Data wrangling is a process of making raw data ready for data analysis.In this process the following tasks will be done<br>

- __Column wise operation__<br>
 - Remove unimportant columns.<br>
 - All columns with that has the text 'url' will be removed.<br>
 - All columns that have constant values will be removed.<br>
 - All columns with missing values more than 40% will be removed<br>
 - Dollar sign and percent sign will be remved from numerical data<br>
 - Text/commnet variables will be removed for this step(some of them will be added later with NLP process/ sentimental analysis<br>
 - Other variables will be individually inspected and will be hand picked to the model.<br>
 - Only 'availability_30' will be selected and the other 'availability' columns will be removed.<br>
- __Row wise operation__<br>
 - If columns have more than 20% and less than 40% of missing, their respective rows will be removed.<br>
 - If columns have less than 20% missing values, their respective rows will be imputed. Numerical columns will pick the mean of the available values and categorical will pick a value that is most frequent.<br>
 - The columnd 'security_deposit' with missing values will be considered as no security deposit is needed to book. as a result zero will be imputed to missing security deposit.<br>
- __Calcuated fields__<br>
 - The dates 'host_since' and 'calendar_last_scraped' will be used to calculate the 'host_experience' and will be removed.<br>
 - The dates 'first_review' and 'last_review' will be used to calculate 'review_duration' and will ber removed.<br>
- Transformation<br>
 - The column 'amenities' will be vectorized to columns 'parking','lounge', 'breakfast', 'beachfront', 'dryer', 'fittness'.<br>
- __Data consistency__<br>
 - The 'availability_30' column values should have values of range between 0 and 30 days. And also it also should be checked agains 'host_experience' column. host experience should be greater than availability.<br><br>
 
- __Exploratory Data Analysis__<br>
For data explotation purpose, I will use the following graphs. <br>
- __'jointplot':__ To show how price and availabilit are distributed against each other.<br>
- __'boxplot':__ To see how price and availability change with property type and room type.<br>
- __'distplot':__ To see how price and availability are distributed for different cancellation policies.<br><br>

- __Predictive Modeling__<br>

I will use 5 learner models with differnt parameters to achieve maximum peroformance.
Price is continuous variable as a result I will use a regression technique and will use RMSE as comparison metrics.<br>
Availablity will be transformed to categorical variable 0 for non availability and 1 other wise.I will use classification technique will choose accuracy as comparison technique.<br><br>

- __Finally__ <br>
To achieve maximum performance of prediction, a good domaing knowledge is very important. I tried to investigate the nature of data both from internet and my intuition. I strongly believe if more time was given on the variable selection, handling the missing values, and some calculated field operation would give an improved result. And also, some spatial analysis could be done. For example given the date and location of the data, we could see price and availability change for different places during summer and winter.

 

 

### Libraries

In [2]:
#!pip install sklearn_pandas
import pandas as pd
import string
import numpy as np
import re
from sklearn_pandas import CategoricalImputer
from datetime import datetime
pd.set_option("display.max_columns", 999)

import warnings
warnings.filterwarnings('ignore')

### Load Data

In [10]:
df=pd.read_csv('listings_ab.csv',low_memory=False)


In [5]:
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2818,https://www.airbnb.com/rooms/2818,20190808195507,2019-08-09,Quiet Garden View Room & Super Fast WiFi,Quiet Garden View Room & Super Fast WiFi,I'm renting a bedroom (room overlooking the ga...,Quiet Garden View Room & Super Fast WiFi I'm r...,none,"Indische Buurt (""Indies Neighborhood"") is a ne...",From week 38 to week 47 maintenance work to th...,The neighbourhood is well served by 24 hours p...,NaN,NaN,Please: - Leave your shoes in the entrance - ...,NaN,NaN,https://a0.muscache.com/im/pictures/10272854/8...,NaN,3159,https://www.airbnb.com/users/show/3159,Daniel,2008-09-24,"Amsterdam, Noord-Holland, The Netherlands","Upon arriving in Amsterdam, one can imagine as...",within an hour,100%,NaN,t,https://a0.muscache.com/im/users/3159/profile_...,https://a0.muscache.com/im/users/3159/profile_...,Indische Buurt,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,f,"Amsterdam, North Holland, Netherlands",Indische Buurt,Oostelijk Havengebied - Indische Buurt,NaN,Amsterdam,North Holland,NaN,Amsterdam,"Amsterdam, Netherlands",NL,Netherlands,52.36575,4.94142,f,Apartment,Private room,2,1.5,1.0,2.0,Real Bed,"{Internet,Wifi,""Paid parking off premises"",""Bu...",NaN,$59.00,NaN,"$1,500.00",$200.00,$60.00,1,$20.00,3,15,3,3,15,15,3.0,15.0,today,t,8,18,43,89,2019-08-09,267,30,2009-03-30,2019-08-05,98.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,"{Amsterdam,"" NL Zip Codes 2"","" Amsterdam"","" NL""}",t,f,strict_14_with_grace_period,f,f,1,0,1,0,2.12
1,20168,https://www.airbnb.com/rooms/20168,20190808195507,2019-08-09,Studio with private bathroom in the centre 1,"Cozy studio on your own private floor, 100% in...",For those who like all facets of city life. In...,"Cozy studio on your own private floor, 100% in...",none,Located just in between famous central canals....,Check-in time from 2pm till 10pm (But it doesn...,No need to use any transport! All is within a ...,NaN,"No curfew, free entrance 27/7 with your own ke...",This studio/room takes entire floor and has it...,NaN,NaN,https://a0.muscache.com/im/pictures/69979664/3...,NaN,59484,https://www.airbnb.com/users/show/59484,Alexander,2009-12-02,"Amsterdam, Noord-Holland, The Netherlands",Secondary phone nr. + (Phone number hidden by ...,within an hour,100%,NaN,f,https://a0.muscach

In [13]:
cnt=df['neighbourhood'].value_counts()

In [18]:
cnt.rank()

Oud-West                               44.0
Oud-Zuid                               43.0
Bos en Lommer                          42.0
Grachtengordel                         41.0
Indische Buurt                         40.0
Oosterparkbuurt                        39.0
Jordaan                                38.0
De Pijp                                37.0
Frederik Hendrikbuurt                  36.0
Rivierenbuurt                          35.0
Amsterdam Centrum                      34.0
Watergraafsmeer                        33.0
Oost                                   32.0
Spaarndammer en Zeeheldenbuurt         31.0
Westelijke Eilanden                    30.0
Slotervaart                            29.0
Osdorp                                 28.0
Hoofddorppleinbuurt                    27.0
Overtoomse Veld                        26.0
IJplein en Vogelbuurt                  25.0
Nieuwmarkt en Lastage                  24.0
Weesperbuurt en Plantage               23.0
Buitenveldert-West              

### All the columns with the name that has 'url' are not important. They need to be removed

In [3]:
df= df.set_index('id')
url_cols=df.filter(like='url', axis=1)
df=df.drop(url_cols.columns,axis=1)
df.shape

(20677, 97)

### Constant columns are not important for analysis.

In [4]:
df = df.loc[:,df.apply(pd.Series.nunique) != 1]
df.shape

(20677, 90)

### Columns with more than 40% of missing are removed

In [5]:
miss_ratio = df.isnull().sum()/len(df)
miss_40pct= df.columns[miss_ratio>.4]
df = df.drop(miss_40pct, axis=1)
df.shape

(20677, 79)

### Remove dollar and percentage signs from numerical data

In [6]:
dolarPercent_cols = ['price','cleaning_fee', 'security_deposit','extra_people']

def removeDollarAndPercent(x):   
    try:
        return float(re.sub(r'[\$\%]','',x))
    except:
        return np.nan
    


In [7]:
for col in dolarPercent_cols:
    df[col] = df[col].apply(lambda x: removeDollarAndPercent(x))
df[dolarPercent_cols].head()

,price,cleaning_fee,security_deposit,extra_people
id,,,,
2818,59.0,60.0,200.0,20.0
20168,80.0,NaN,NaN,0.0
25428,125.0,40.0,300.0,10.0
27886,150.0,0.0,0.0,0.0
28871,75.0,NaN,NaN,0.0


### Remove text columns, we will return to this when we do NLP 

In [8]:
df =df.drop(['last_scraped', 'name', 'summary', 'space', 'description',
       'neighborhood_overview','transit', 'interaction', 'house_rules',
       'host_id'],axis=1)
df.shape

(20677, 69)

### Investigate each column by taking subsets at a time and add/remove columns

In [9]:
df=df.drop([ 'host_name','host_location', 'host_neighbourhood', 'host_listings_count', 'host_verifications',
       'host_has_profile_pic',  'street',    'neighbourhood','neighbourhood_cleansed', 'city', 'state', 'zipcode',
       'market', 'smart_location', 'latitude'],axis=1)
df.shape

(20677, 54)

In [10]:
df = df.drop(['longitude', 'is_location_exact', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm','jurisdiction_names'],axis=1)
df.shape

(20677, 45)

### Calculated column: merge 'host_since' and 'calendar_last_scraped' as 'host experience'

In [11]:
df=df.dropna(subset=['host_since'])
df['host_experience'] = df[["host_since","calendar_last_scraped"]].apply(lambda d: 
                                                       (datetime.strptime(str(d[1]),'%Y-%m-%d') 
                                                        - datetime.strptime(str(d[0]), '%Y-%m-%d')).days, axis=1 )
df.drop(columns=["host_since","calendar_last_scraped"], axis=1, inplace=True)
df.shape

(20665, 44)

### both last_review and first_review have na the same row. replace with the same date to avoid nan problem

In [12]:
# set null values of both columns to same date, and the  will result in 0 duration
df['last_review']=np.where(df.last_review.isnull(),'2009-03-30',df.last_review)
df['first_review']=np.where(df.first_review.isnull(),'2009-03-30',df.first_review)


In [13]:
df['review_duration'] = df[["first_review","last_review"]].apply(lambda d: 
                                                       (datetime.strptime(str(d[1]),'%Y-%m-%d') 
                                                        - datetime.strptime(str(d[0]), '%Y-%m-%d')).days, axis=1 )
df=df.drop(["first_review","last_review"],axis=1)
df.shape

(20665, 43)

### 'availability_30'  and 'host_experience'  consistency.
the 'availability_60', 'availability_90', and 'availability_365' will be removed
'availability_30' will be verified for consistency  should be greater thatn 0 and less or equal 30
'host_experience'  should be greater than 30

In [14]:
df.drop(columns=['availability_60','availability_90','availability_365'], axis=1, inplace=True)
df.shape

(20665, 40)

In [15]:
df = df.query("(host_experience > 30) and (bedrooms > 0) and (availability_30 < 30)")
df.shape

(19192, 40)

In [16]:
#df['availability_30']=df['availability_30'].apply(lambda x: 0 if x==0 else 1)


In [17]:
df['availability_30'].mean()

3.427573989162151

###  Common amenities(from internet research) were listed and copared against the host amenities

In [18]:
# function for checking availablilty in the list and assign value
def amenitiesRefined(list, key):
    if key in map(str.lower,list):
        return 1
    else:
        return 0    
# Convert the text in the column to list , for parsing purpose
df['amenities'] = df['amenities'].apply(lambda x : re.findall(r"\w+", x) )

amenities = ['parking','lounge', 'breakfast', 'beachfront', 'dryer', 'fittness']

# 
for amenity in amenities:
    df[amenity] = df['amenities'].apply(lambda l: amenitiesRefined(l, amenity) )
df=df.drop('amenities', axis=1)
df.shape

(19192, 45)

###  Remove rows that have less than 20 % Missing

In [19]:
na_cols=((df.isna().sum()/df.shape[0])*100)
na_cols=na_cols[na_cols.values >20].index
df=df.dropna(subset=na_cols)
df.shape

(13067, 45)

### If security deposit is missing, means no security deposit. replace missing values by zero.

In [20]:
df['security_deposit'].fillna('0', inplace=True)

### Split dataset into Categorical and Numaric columns for imputation purpose

In [21]:

categorical_cols = df.loc[:,df.dtypes==np.object]
#categorical_cols=categorical_cols.drop(['first_review','last_review'],axis=1)
numerical_cols = df.loc[:,df.dtypes!=np.object]
#Drop ID
#numerical_cols=numerical_cols.drop('id',axis=1)

### Replace missing values for numeric data with the mean of the available data

In [22]:
# percentage of missing in each columns
na_cols=((numerical_cols.isna().sum()/numerical_cols.shape[0])*100)
# Columns with missing greater than 5%: NOTE we handled columns with 20% and above. Upper bound here is 20%
na_cols=na_cols[na_cols.values >5].index
numerical_cols[na_cols]=numerical_cols[na_cols].fillna(numerical_cols[na_cols].mean().iloc[0])

### Replace the missing categorical data with the most frequent data

In [23]:
for cols in categorical_cols:
    imputer = CategoricalImputer()
    categorical_cols[cols]=imputer.fit_transform(categorical_cols[cols])
categorical_cols.head()

,host_is_superhost,host_identity_verified,property_type,room_type,bed_type,calendar_updated,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
id,,,,,,,,,,
2818,t,f,Apartment,Private room,Real Bed,today,t,strict_14_with_grace_period,f,f
25428,f,f,Apartment,Entire home/apt,Real Bed,today,f,strict_14_with_grace_period,f,f
27886,t,t,Houseboat,Private room,Real Bed,today,t,strict_14_with_grace_period,f,f
38266,f,f,Apartment,Entire home/apt,Futon,5 months ago,f,strict_14_with_grace_period,f,f
42970,t,f,Bed and breakfast,Private room,Real Bed,today,t,strict_14_with_grace_period,f,t


### create csv file on the working directory to save our clean data set for analysis use

In [24]:
# df= pd.merge(numerical_cols,categorical_cols)
mergedDf = numerical_cols.merge(categorical_cols, left_index=True, right_index=True)
mergedDf=mergedDf.dropna()
mergedDf.to_csv('clean_listings.csv')
